In [1]:
import csv
import time
from tools.yahoo_api import get_key_stats

In [2]:
import pandas as pd
import plotly.plotly as py
import numpy as np
import collections
import json
from plotly.graph_objs import *
import matplotlib.pyplot as plt

In [3]:
def wait(seconds):
    time.sleep(seconds)

In [4]:
filename = 'sp500'
filename1= '2016-01-19'
filename2= '2016-08-23'

In [5]:
def write_stats(tickers, filename):
    print 'writing to file...'
    writer = csv.writer(open(filename, 'wb'))
    # write headers
    writer.writerow(tickers[tickers.keys()[0]].keys())
    for ticker in tickers:
        writer.writerow(tickers[ticker].values())
    print 'done'

In [6]:
def get_stats_for(tickers):
    for ticker in tickers.keys():
        print 'getting stats for ', ticker
        try:
            names, stats = get_key_stats(ticker)
            # print names, stats
            wait(0.2) # being a good citizen
            for name, stat in zip(names,stats):
                tickers[ticker][name] = stat
        # keep ticker stats only if all stats are valid
        except Exception, e:
            tickers.pop(ticker, None)
            print 'failed to save to dict ', str(e)
    print 'done'
    return tickers

In [7]:
def get_tickers_dict(filename):
    tickers = {}
    print 'filename', filename
    try:
        f = open(filename,'r').read()
        split_file = f.split('\n')
        for line in split_file:
            if line:
                split_line = line.split(',')
                tickers[split_line[0]] = {'name': split_line[1],'industry': split_line[2]}
        return tickers
    except Exception, e:
        print 'failed in get_tickers_dict', str(e)
    print 'done'

In [ ]:
tickers = get_tickers_dict(filename + '.txt')

In [ ]:
stats = get_stats_for(tickers)
# print stats

In [ ]:
write_stats(stats, filename + '.csv')

In [8]:
with open('tools/plotly_credentials.json', 'r') as creds:
        credentials = json.load(creds)

In [9]:
py.sign_in(credentials['plotly']['username'],
               credentials['plotly']['key'])

In [5]:
initial_state = pd.read_csv(filename1 + '.csv')
initial_state = initial_state.set_index('name')
print initial_state.head()

                  pb                industry    peg            mc
name                                                             
Allergan plc    1.61             Health Care   2.24  1.174600e+11
EOG Resources   2.64                  Energy -47.20  3.629000e+10
Campbell Soup  11.86        Consumer Staples   3.95  1.694000e+10
Qorvo           0.93  Information Technology   0.67  5.800000e+09
Nordstrom       6.29  Consumer Discretionary   2.92  8.830000e+09


In [6]:
final_state = pd.read_csv(filename2 + '.csv')
final_state = final_state.set_index('name')
print final_state.head()

                         mc                industry     pb         r   peg
name                                                                      
Allergan plc   1.002600e+11             Health Care   1.40 -0.132218  1.34
EOG Resources  5.022000e+10                  Energy   4.16  0.501949 -0.20
Campbell Soup  1.866000e+10        Consumer Staples  11.30  0.123233  2.71
Qorvo          7.170000e+09  Information Technology   1.42  0.543951  0.83
Nordstrom      9.170000e+09  Consumer Discretionary   9.64  0.165682  3.18


In [7]:
returns = final_state[['r']]
print returns.head()

                      r
name                   
Allergan plc  -0.132218
EOG Resources  0.501949
Campbell Soup  0.123233
Qorvo          0.543951
Nordstrom      0.165682


In [8]:
grid_by = initial_state[['industry', 'pb', 'peg']]
print grid_by.head()

                             industry     pb    peg
name                                               
Allergan plc              Health Care   1.61   2.24
EOG Resources                  Energy   2.64 -47.20
Campbell Soup        Consumer Staples  11.86   3.95
Qorvo          Information Technology   0.93   0.67
Nordstrom      Consumer Discretionary   6.29   2.92


In [9]:
complete = pd.concat([returns, grid_by], axis=1, join='inner').dropna()
print complete.head()
print complete.tail()

                      r                industry     pb    peg
name                                                         
Allergan plc  -0.132218             Health Care   1.61   2.24
EOG Resources  0.501949                  Energy   2.64 -47.20
Campbell Soup  0.123233        Consumer Staples  11.86   3.95
Qorvo          0.543951  Information Technology   0.93   0.67
Nordstrom      0.165682  Consumer Discretionary   6.29   2.92
                                  r                industry    pb   peg
name                                                                   
Intuitive Surgical         0.250624             Health Care  4.80  1.96
Pfizer                     0.174892             Health Care  2.83  3.19
NIKE                       0.027921  Consumer Discretionary  7.71  2.24
Principal Financial Group  0.294218              Financials  1.12  3.40
Total System Services      0.081990  Information Technology  4.43  1.45


In [10]:
companiesNB = len(complete)
print companiesNB

451


In [11]:
# complete = complete[(complete > 0).all(1)]

In [12]:
companiesNB = len(complete)
print companiesNB

451


In [31]:
xaxisMAX = 5;
yaxisMAX = 5;
step = 0.1;

z = pd.DataFrame(np.zeros((100, 100)))
count = pd.DataFrame(np.zeros((100, 100)))

In [32]:
for index, row in complete.iterrows():
    x = (round(row['pb'],1))
    y = (round(row['peg'],1))
    if (x < xaxisMAX and y < yaxisMAX and x > -xaxisMAX and y > -yaxisMAX):
        print index, x, y
        x = int(x * 10) + 50
        y = int(y * 10) + 50      
        print index, x, y
        z[x][y] = z[x][y] + row['r']
        count[x][y] = count[x][y] + 1.0
        print index, x, y, z[x][y], count[x][y]


Allergan plc 1.6 2.2
Allergan plc 66 72
Allergan plc 66 72 -0.132218402995 1.0
Qorvo 0.9 0.7
Qorvo 59 57
Qorvo 59 57 0.543951459246 1.0
Borg Warner 1.8 1.1
Borg Warner 68 61
Borg Warner 68 61 0.144804211737 1.0
Waste Management 4.4 2.6
Waste Management 94 76
Waste Management 94 76 0.258031053055 1.0
Endo International plc 2.0 0.9
Endo International plc 70 59
Endo International plc 70 59 -0.550702620585 1.0
Goodyear Tire & Rub 1.8 0.8
Goodyear Tire & Rub 68 58
Goodyear Tire & Rub 68 58 0.0747847818366 1.0
Roper Technologies 3.3 2.4
Roper Technologies 83 74
Roper Technologies 83 74 0.0477039826793 1.0
Genl Electric 2.6 2.8
Genl Electric 76 78
Genl Electric 76 78 0.113320442392 1.0
Genl Dynamics 3.7 1.4
Genl Dynamics 87 64
Genl Dynamics 87 64 0.208292352779 1.0
Varian Medical Systems 4.3 2.7
Varian Medical Systems 93 77
Varian Medical Systems 93 77 0.251909369189 1.0
General Motors 1.4 0.3
General Motors 64 53
General Motors 64 53 0.112627788266 1.0
Snap-On 3.7 1.9
Snap-On 87 69
Snap-On 8

In [33]:
z.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
count.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
z = np.divide(z, count)
z.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
z[np.isnan(z)] = 0
z.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
data = [
    Contour(
        z=z.as_matrix()
    )
]

In [38]:
layout = Layout(
    title='Value Investing with Price to Book and Price to Earnings Growth',
    xaxis=dict(
        title='PB',
        range=[0, 2],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='PEG',
        range=[0, 2],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

In [39]:
fig = Figure(data=data, layout=layout)

In [51]:
py.iplot(fig, filename='compare ' + filename1 + ' and ' + filename2)